In [70]:
import numpy as np
import pandas as pd
import nltk

In [71]:
#Download requirements
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [72]:
data = pd.read_csv('Data/digikala_comment.csv')
data

,comment
0,نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ق...
1,چند ماهی میشه که گرفتمش‌. برای برنامه نویسی و ...
2,پراید ستون جدید
3,اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره ...
4,گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ا...
...,...
235,پوشش دهی صفر.اصلا پیشنهاد نمیکنم
236,نصب این فن خیلی راحته و دردسر زیادی نداره درض...
237,بی کیفیت
238,سلام ٬ چندماهی میشه این پاور بانک رو تهیه کردم...


In [73]:
# Sentence Tokenizing
from nltk.tokenize import sent_tokenize

def sentence_tokenizing(text):
    return sentence_tokenizing(text)

data['comment'] = data.apply(lambda text:sent_tokenize(text['comment']), axis=1)
data

,comment
0,[نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ...
1,"[چند ماهی میشه که گرفتمش‌., برای برنامه نویسی ..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ...
...,...
235,[پوشش دهی صفر.اصلا پیشنهاد نمیکنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره در...
237,[بی کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [74]:
import re
import string

def clean_fa_text(text):
    #removing english characters and signs
    text = ''.join([i for i in text if not ((65 <= ord(i) <91)
                                            or (97 <= ord(i) < 123)
                                            or (48 <= ord(i) < 58))])

    #removing nbsp
    text_list = []
    for char in text:
        if ord(char) == 160:
            text_list.append(' ')
            continue
        text_list.append(char)
    text = ''.join(text_list)

    # removing sign
    text = ''.join([i for i in text if ord(i) not in [33, 34, 35, 36, 37, 38,
                                                      39, 40, 41, 42, 43, 44,
                                                      45, 46, 47, 58, 59, 60,
                                                      61, 62, 63, 64, 91, 92,
                                                      93, 94, 95, 96, 123, 124,
                                                      125, 126, 1548, 1567]])
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def apply_clean(list_sentence: list):
    new_list = []
    for sent in list_sentence:
        new_list.append(clean_fa_text(sent))
    return new_list

data['comment'] = data['comment'].apply(apply_clean)
data

,comment
0,[نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشن...
1,"[چند ماهی میشه که گرفتمش‌, برای برنامه نویسی و..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی سیپیو و دوربین و رمش از این خیل...
...,...
235,[پوشش دهی صفراصلا پیشنهاد نمیکنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره در...
237,[بی کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [75]:
#text normalizing
from hazm import Normalizer

def apply_normalizer(list_sentence: list):
    new_list = []
    norm = Normalizer()
    for sent in list_sentence:
        new_list.append(norm.normalize(sent))
    return new_list

data['comment'] = data['comment'].apply(apply_normalizer)
data

,comment
0,[نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشن...
1,"[چند ماهی میشه که گرفتمش, برای برنامه نویسی و ..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی سیپیو و دوربین و رمش از این خیلی ...
...,...
235,[پوشش دهی صفراصلا پیشنهاد نمی‌کنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره درض...
237,[بی‌کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [80]:
#concat all sentences
def append_all(data, column: str):
    all_sentences = []
    for sent in data[column]:
        all_sentences.extend(sent)
    return all_sentences

all_sentences = append_all(data, 'comment')
all_sentences

['نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشنگهتنها مشکلش بندهای ضعیفش هست که باعث میشه استحکام چندانی نداشنه باشه',
 'چند ماهی میشه که گرفتمش',
 'برای برنامه نویسی و کارای گرافیکی ازش استفاده می\u200cکنم',
 'واقعا از هر لحاظ بگین عالیه',
 'پراید ستون جدید',
 'اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره که با روشن شدن گوشی بیشتر هم میشه',
 'و نکته دیگه اینکه به خاطر این\u200cکه اطرافش یه کوچلو خمیده هست گلس بعد یه مدتی جدا مشیه',
 'ولی در کل با این قیمت بهترین گوشی هست و همه چی داره از دوربین گرفته تا رم و سی پی یو و گرافیک و حسگر\u200cهای مختلف و خیلی چیزای دیگه',
 'گوسی هو اوی سیپیو و دوربین و رمش از این خیلی بهتره خودتون میتونین برین تمام مقایسه\u200cهای و این گوشیو ببینین',
 'چادر سبک و زیباییه دوختشم عالیه جنسشم خیلی خوبه چروک نمیشه',
 'خیلی خوبه',
 'حتما پیشنهاد می\u200cکنم',
 'من چند روزیه این خردکن رو گرفتم',
 'واقعا کارکرد خوبی دارهطراحی ظاهر خردکن هم به نسبت بقیه خردکن\u200cها خیلی زیباتره',
 'تیغه\u200cهای بسیار تیزی داره که اگه موقع شستن حواستون نباشه خیلی راحت دستتون ر

In [92]:
#splite sentence wrt words

def split_sentences(data: list):
    dat=[]
    for i in range(len(data)):
        for word in data[i].split():
            dat.append(word)
    return dat

all_words = split_sentences(all_sentences)
all_words

['نسبت',
 'به',
 'قیمتش',
 'ارزش',
 'خرید',
 'دارهجاداره',
 'طراحیش',
 'قشنگهتنها',
 'مشکلش',
 'بندهای',
 'ضعیفش',
 'هست',
 'که',
 'باعث',
 'میشه',
 'استحکام',
 'چندانی',
 'نداشنه',
 'باشه',
 'چند',
 'ماهی',
 'میشه',
 'که',
 'گرفتمش',
 'برای',
 'برنامه',
 'نویسی',
 'و',
 'کارای',
 'گرافیکی',
 'ازش',
 'استفاده',
 'می\u200cکنم',
 'واقعا',
 'از',
 'هر',
 'لحاظ',
 'بگین',
 'عالیه',
 'پراید',
 'ستون',
 'جدید',
 'اقا',
 'همه',
 'چیش',
 'خوبه',
 'فقط',
 'از',
 'پایین',
 'زیاد',
 'حاشیه',
 'داره',
 'که',
 'با',
 'روشن',
 'شدن',
 'گوشی',
 'بیشتر',
 'هم',
 'میشه',
 'و',
 'نکته',
 'دیگه',
 'اینکه',
 'به',
 'خاطر',
 'این\u200cکه',
 'اطرافش',
 'یه',
 'کوچلو',
 'خمیده',
 'هست',
 'گلس',
 'بعد',
 'یه',
 'مدتی',
 'جدا',
 'مشیه',
 'ولی',
 'در',
 'کل',
 'با',
 'این',
 'قیمت',
 'بهترین',
 'گوشی',
 'هست',
 'و',
 'همه',
 'چی',
 'داره',
 'از',
 'دوربین',
 'گرفته',
 'تا',
 'رم',
 'و',
 'سی',
 'پی',
 'یو',
 'و',
 'گرافیک',
 'و',
 'حسگر\u200cهای',
 'مختلف',
 'و',
 'خیلی',
 'چیزای',
 'دیگه',
 'گوسی',
 'هو',
 'او

In [ ]:
# bigram model

def create_bigram(data):
    list_of_bigrams = []
    bigram_counts = {}
    unigram_counts = {}

    for i in range(len(data)-1):
        if i < len(data) - 1 and data[i+1]:
            list_of_bigrams.append((data[i], data[i + 1]))

            if (data[i], data[i+1]) in bigram_counts:
                bigram_counts[(data[i], data[i + 1])] += 1
            else:
                bigram_counts[(data[i], data[i + 1])] = 1

        if data[i] in unigram_counts:
            unigram_counts[data[i]] += 1
        else:
            unigram_counts[data[i]] = 1

    return list_of_bigrams, unigram_counts, bigram_counts


def bigram_probability(list_of_bigrams, unigram_counts, bigram_counts):
    prob_list = {}
    for bigram in list_of_bigrams:
        word1 = bigram[0]
        prob_list[bigram] = (bigram_counts.get(bigram))/(unigram_counts.get(word1))
    return prob_list


list_of_bigrams, unigram_counts, bigram_counts = create_bigram(all_words)
bigram_prob = bigram_probability(list_of_bigrams, unigram_counts, bigram_counts)

In [94]:
#trigram model

def create_trigram(data):
    list_of_trigrams = []
    trigram_counts = {}
    unigram_counts = {}

    for i in range(len(data)-2):
        if i < len(data) - 2 and data[i+2]:
            list_of_trigrams.append((data[i], data[i + 1], data[i + 2]))

            if (data[i], data[i + 1], data[i + 2]) in trigram_counts:
                trigram_counts[(data[i], data[i + 1], data[i + 2])] += 1
            else:
                trigram_counts[(data[i], data[i + 1], data[i + 2])] = 1

        if data[i] in unigram_counts:
            unigram_counts[data[i]] += 1
        else:
            unigram_counts[data[i]] = 1

    return list_of_trigrams, unigram_counts, trigram_counts


def trigram_probability(list_of_bigrams, unigram_counts, bigram_counts):
    prob_list = {}
    for bigram in list_of_bigrams:
        word1 = bigram[0]
        prob_list[bigram] = (bigram_counts.get(bigram))/(unigram_counts.get(word1))
    return prob_list

list_of_trigrams, unigram_counts, trigram_counts = create_trigram(all_words)
trigram_prob = trigram_probability(list_of_trigrams, unigram_counts, trigram_counts)

In [95]:
trigram_prob

{('نسبت', 'به', 'قیمتش'): 0.34615384615384615,
 ('به', 'قیمتش', 'ارزش'): 0.006578947368421052,
 ('قیمتش', 'ارزش', 'خرید'): 0.05555555555555555,
 ('ارزش', 'خرید', 'دارهجاداره'): 0.09090909090909091,
 ('خرید', 'دارهجاداره', 'طراحیش'): 0.037037037037037035,
 ('دارهجاداره', 'طراحیش', 'قشنگهتنها'): 1.0,
 ('طراحیش', 'قشنگهتنها', 'مشکلش'): 0.5,
 ('قشنگهتنها', 'مشکلش', 'بندهای'): 1.0,
 ('مشکلش', 'بندهای', 'ضعیفش'): 1.0,
 ('بندهای', 'ضعیفش', 'هست'): 1.0,
 ('ضعیفش', 'هست', 'که'): 1.0,
 ('هست', 'که', 'باعث'): 0.018518518518518517,
 ('که', 'باعث', 'میشه'): 0.010810810810810811,
 ('باعث', 'میشه', 'استحکام'): 0.09090909090909091,
 ('میشه', 'استحکام', 'چندانی'): 0.02857142857142857,
 ('استحکام', 'چندانی', 'نداشنه'): 1.0,
 ('چندانی', 'نداشنه', 'باشه'): 1.0,
 ('نداشنه', 'باشه', 'چند'): 1.0,
 ('باشه', 'چند', 'ماهی'): 0.03571428571428571,
 ('چند', 'ماهی', 'میشه'): 0.0625,
 ('ماهی', 'میشه', 'که'): 1.0,
 ('میشه', 'که', 'گرفتمش'): 0.02857142857142857,
 ('که', 'گرفتمش', 'برای'): 0.005405405405405406,
 ('گرفت